<a href="https://colab.research.google.com/github/SuperCoolCucumber/CompLing/blob/main/MT_homework11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 11. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). 

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Подсказка: модель может предсказывать батчами.


In [ ]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-uk/opus.en-uk-train.uk
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-uk/opus.en-uk-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-uk/opus.en-uk-test.uk
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-uk/opus.en-uk-test.en

--2022-05-24 13:50:45--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-uk/opus.en-uk-train.uk
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62317727 (59M)
Saving to: ‘opus.en-uk-train.uk’

opus.en-uk-train.uk 100%[===================>]  59.43M  20.9MB/s    in 2.8s    

2022-05-24 13:50:49 (20.9 MB/s) - ‘opus.en-uk-train.uk’ saved [62317727/62317727]

--2022-05-24 13:50:49--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-uk/opus.en-uk-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35911829 (34M)
Saving to: ‘opus.en-uk-train.en’

opus.en-uk-train.en 100%[===================>]  34.25M  18.7MB/s    in 1.8s    

2022-05-24 13:50:51 (18.7 MB/s) - ‘opus.e

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
text = open('opus.en-uk-train.uk').read().replace('\xa0', ' ')
f = open('opus.en-ru-train.uk', 'w')
f.write(text)
f.close()

In [ ]:
en_sents = open('opus.en-uk-train.en').read().splitlines()
uk_sents = open('opus.en-uk-train.uk').read().splitlines()

In [ ]:
len(en_sents)

1000000

In [ ]:
len(uk_sents)

1000000

In [ ]:
en_sents[-1]

'Yeah. It really sucks.'

In [ ]:
uk_sents[-1]

'Да.Это хреново.'

In [ ]:
tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_en.train(files=["opus.en-uk-train.en"], trainer=trainer_en)

tokenizer_uk = Tokenizer(BPE())
tokenizer_uk.pre_tokenizer = Whitespace()
trainer_uk = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_uk.train(files=["opus.en-uk-train.uk"], trainer=trainer_uk)

In [ ]:

tokenizer_en.save('tokenizer_en')
tokenizer_uk.save('tokenizer_uk')

In [ ]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_uk = Tokenizer.from_file("tokenizer_uk")

In [ ]:
def encode(text, tokenizer, max_len):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[SEP]')]

In [ ]:
PAD_IDX = tokenizer_uk.token_to_id('[PAD]')
PAD_IDX

3

In [ ]:
max_len_en, max_len_uk = 50, 50

In [ ]:
X_en = [encode(t, tokenizer_en, max_len_en) for t in en_sents]
X_uk = [encode(t, tokenizer_uk, max_len_uk) for t in uk_sents]

In [ ]:
len(X_en)

1000000

In [ ]:
len(X_uk)

1000000

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_en, texts_uk):
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, padding_value=PAD_IDX)
        
        self.texts_uk = [torch.LongTensor(sent) for sent in texts_uk]
        self.texts_uk = torch.nn.utils.rnn.pad_sequence(self.texts_uk, padding_value=PAD_IDX)

        self.length = len(texts_en)
    
    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[:, index]
        ids_uk = self.texts_uk[:, index]

        return ids_en, ids_uk

In [ ]:
X_en_train, X_en_valid, X_uk_train, X_uk_valid = train_test_split(X_en, X_uk, test_size=0.05)

In [ ]:
training_set = Dataset(X_en_train, X_uk_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=True, )

In [ ]:
valid_set = Dataset(X_en_valid, X_uk_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=200, shuffle=True)

### Transformer (from [pytorch tutorial](https://pytorch.org/tutorials/beginner/transformer_tutorial.html))

In [ ]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 150):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size, 
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         print('pos inp')
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         print('pos dec')
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
#         print('pos out')
        x = self.generator(outs)
#         print('gen')
        return x

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
# During training, we need a subsequent word mask that will prevent model to look into the future words when making predictions. We will also need masks to hide source and target padding tokens. Below, let’s define a function that will take care of both.

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
from time import time
def train(model, iterator, optimizer, criterion, print_every=500):
    
    epoch_loss = []
    ac = []
    
    model.train()  

    for i, (texts_en, texts_uk) in enumerate(iterator):
        texts_en = texts_en.T.to(DEVICE) 
        texts_uk = texts_uk.T.to(DEVICE) 
        

        texts_uk_input = texts_uk[:-1, :]
        

        (texts_en_mask, texts_uk_mask, 
        texts_en_padding_mask, texts_uk_padding_mask) = create_mask(texts_en, texts_uk_input)
        logits = model(texts_en, texts_uk_input, texts_en_mask, texts_uk_mask,
                       texts_en_padding_mask, texts_uk_padding_mask, texts_en_padding_mask)
        optimizer.zero_grad()
        
        texts_uk_out = texts_uk[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_uk_out.reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        
        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')
        
    return np.mean(epoch_loss)



def evaluate(model, iterator, criterion):
    
    epoch_loss = []
    epoch_f1 = []

    model.eval()
    for i, (texts_en, texts_uk) in enumerate(iterator):
        texts_en = texts_en.T.to(DEVICE) 
        texts_uk = texts_uk.T.to(DEVICE)
        texts_uk_input = texts_uk[:-1, :]

        
        (texts_en_mask, texts_uk_mask, 
        texts_en_padding_mask, texts_uk_padding_mask) = create_mask(texts_en, texts_uk_input)
        logits = model(texts_en, texts_uk_input, texts_en_mask, texts_uk_mask,
                       texts_en_padding_mask, texts_uk_padding_mask, texts_en_padding_mask)
        
        texts_uk_out = texts_uk[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_uk_out.reshape(-1))
        epoch_loss.append(loss.item())
        
        
    return np.mean(epoch_loss)

In [ ]:
torch.manual_seed(0)

EN_VOCAB_SIZE = tokenizer_en.get_vocab_size()
UK_VOCAB_SIZE = tokenizer_uk.get_vocab_size()

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, EN_VOCAB_SIZE, UK_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
#torch.save(transformer, 'model')

In [ ]:
#transformer = torch.load('model').to(DEVICE)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 10

losses = []

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(transformer, training_generator, optimizer, loss_fn)
    end_time = timer()
    val_loss = evaluate(transformer, valid_generator, loss_fn)
    
    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    losses.append(val_loss)
        
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

Loss: 7.4664886770248415;
Loss: 6.881732636451721;
Loss: 6.580780406951904;
Loss: 6.369524511575698;
Loss: 6.2078326839447024;
Loss: 6.077315409819285;
Loss: 5.9663321219853;
Loss: 5.8682837243080135;
Loss: 5.7823254144456655;
First epoch - 4.874063325881958, saving model..
Epoch: 1, Train loss: 5.742, Val loss: 4.874,            Epoch time=1227.709s
Loss: 4.91703074836731;
Loss: 4.880656561374664;
Loss: 4.846615245501201;
Loss: 4.813653960466385;
Loss: 4.780827764892578;
Loss: 4.748732059955597;
Loss: 4.717852898733956;
Loss: 4.689744362950325;
Loss: 4.6613774173524645;
Improved from 4.874063325881958 to 4.2287362213134765, saving model..
Epoch: 2, Train loss: 4.647, Val loss: 4.229,            Epoch time=1234.150s
Loss: 4.294831292629242;
Loss: 4.279468092918396;
Loss: 4.259419092019399;
Loss: 4.238797508478164;
Loss: 4.219801478672028;
Loss: 4.200634664058685;
Loss: 4.181723873751504;
Loss: 4.164667080402374;
Loss: 4.147414542145199;
Improved from 4.2287362213134765 to 3.81835559558

In [ ]:
def translate(text):


    input_ids = [tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(text).ids[:max_len_en] + [tokenizer_en.token_to_id('[SEP]')]
    output_ids = [tokenizer_uk.token_to_id('[CLS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

    (texts_en_mask, texts_uk_mask, 
    texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_uk_mask,
                   texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
    pred = logits.argmax(2).item()

    while pred not in [tokenizer_uk.token_to_id('[SEP]'), tokenizer_uk.token_to_id('[PAD]')]:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

        (texts_en_mask, texts_uk_mask, 
        texts_en_padding_mask, texts_uk_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_uk_mask,
                       texts_en_padding_mask, texts_uk_padding_mask, texts_en_padding_mask)
        pred = logits.argmax(2)[-1].item()

    return (' '.join([tokenizer_uk.id_to_token(i).replace('##', '') for i in output_ids[1:]]))



In [ ]:
translate('Does this function work?')

'Ця функція працює ?'

### BLEU Score

In [ ]:
import nltk

In [ ]:
en_sents_test = open('opus.en-uk-test.en').read().lower().splitlines()
uk_sents_test = open('opus.en-uk-test.uk').read().lower().splitlines()

In [ ]:
translations = []

for i in range(len(en_sents_test)):
  translations.append(translate(en_sents_test[i]))

In [ ]:
bleus = []

for i, t in enumerate(translations):
  reference = tokenizer_uk.encode(t).tokens
  hypothesis = tokenizer_uk.encode(uk_sents_test[i]).tokens

bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  ))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [ ]:
bleus

[0.34772504705825924]


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf 
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара. 

**Backtranslation** применяется при недостатке данных параллельных корпусов, обычно для низкоресурсных языков. Эта техника помогает, когда есть много данных на одном языке и мало на другом. Backtranslation заключается в следующем: на существующих данных обучается система машинного перевода, после чего эта система используется, чтобы перевести данные с языка перевода на исходный язык (поэтому **back**). Дальше переведенные тексты просто добавляются к малоресурному корпусу, чтобы дальше обучать более крупные системы. Таким образом, можно получить качественный машинный перевод даже на малом количестве данных исходного языка.


На данных из семинара пример можно было бы примениить, взяв много текстов на русском и мало на английском. С помощью этих данных обучить трансформер, использовать его для перевода тестовой выборки и добавить эти переведенные тексты в новую тренировочную выборку, увеличив тем самым объем тренировочных данных. Также нужно выровнять новые полученные параллельные корпуса и обучить новый трансформер, тестовую выборку которого можно снова добавлять в новую тренировочную и так далее.